In [1]:
import pandas as pd
import numpy as np
from LoopStructural import GeologicalModel
import pyvista as pv
from LoopStructural.interpolators.supports import P1Unstructured2d, StructuredGrid2D

In [2]:
df = pd.DataFrame(
    [
        [5, 5, 5, 0.1, 0, 0.86, 0],
        [8, 5, 4, 0, 0, 1, 0],
        [2, 5, 4, 0, 0, 1, 0],
        [2, 2, 4, 0, 0, 1, 0],
        [2, 7, 4, 0, 0, 1, 0],
    ],
    columns=["X", "Y", "Z", "nx", "ny", "nz", "val"],
)
df["feature_name"] = "field"

In [3]:
model = GeologicalModel(np.zeros(3), np.ones(3) * 10)
model.data = df
model.create_and_add_foliation("field")
model.update()

  0%|          | 0/1 [00:00<?, ?it/s]

In [29]:
# to get the isosurfaces you can call feature.surfaces([0.]) where the argument is the list of values to isosurface
# the resolution of the isosurfaces is defined by the model.bounding_box
# or you can pass a BoundingBox object to define the area and resolution to isosurface
print(model.bounding_box.nsteps)
model.bounding_box.nelements = 1e6
print(model.bounding_box.nsteps)
surfaces = model["field"].surfaces([0.0])

[47 47 47]
[100 100 100]


In [30]:
surfaces

[Surface(vertices=array([[ 0.        ,  0.        ,  3.74262993],
        [ 0.        ,  0.1010101 ,  3.74420551],
        [ 0.1010101 ,  0.        ,  3.75347908],
        ...,
        [10.        ,  9.7979798 ,  4.4559209 ],
        [10.        ,  9.8989899 ,  4.45808295],
        [10.        , 10.        ,  4.46024461]]), triangles=array([[    2,     1,     0],
        [    3,     1,     2],
        [    3,     4,     1],
        ...,
        [11344, 11242, 11343],
        [11344, 11243, 11242],
        [11345, 11243, 11344]]), normals=array([[ 0.10673615,  0.01550871, -0.99416643],
        [ 0.10828506,  0.01538583, -0.9940008 ],
        [ 0.1068176 ,  0.01683933, -0.99413604],
        ...,
        [ 0.00485526,  0.02136716, -0.9997599 ],
        [ 0.00571292,  0.02136293, -0.99975544],
        [ 0.00668195,  0.02129244, -0.999751  ]], dtype=float32), name='field_0.0', values=array([0., 0., 0., ..., 0., 0., 0.]), properties=None)]

In [31]:
p = pv.Plotter()
p.add_mesh(surfaces[0].vtk)
p.show()

Widget(value="<iframe src='http://localhost:50953/index.html?ui=P_0x1d484165590_6&reconnect=auto' style='width…

In [32]:
mesh = P1Unstructured2d(
    surfaces[0].triangles, surfaces[0].vertices[:, 0:2], surfaces[0].triangles[:, 0:3]
)

In [33]:
structured2d = StructuredGrid2D(
    origin=np.zeros(2), step_vector=np.ones(2), nsteps=np.ones(2, dtype=int) * 11
)
pts = structured2d.barycentre

In [34]:
z = mesh.evaluate_value(pts, surfaces[0].vertices[:, 2])

0 10000 100


In [35]:
# we expect the values to be close to zero depending on the resultion of the interpolation
model["field"].evaluate_value(np.hstack([pts, z[:, np.newaxis]]))

array([-1.07365631e-03, -9.13525658e-04, -6.41173724e-04, -2.51104624e-04,
       -1.76431047e-04, -3.72801064e-04,  4.06762478e-04,  3.52683967e-05,
       -1.24826180e-04, -8.98240755e-05, -1.23952781e-03, -9.85792320e-04,
       -6.29330329e-04, -3.20785042e-04, -2.01820636e-04, -5.30342763e-04,
        1.98802934e-04, -4.98659163e-04, -7.35620858e-05, -7.31201583e-05,
       -1.59995543e-03, -1.69961260e-03, -7.46491487e-04,  1.36760472e-04,
        5.76831216e-04, -9.19725548e-05,  1.99576976e-05, -1.15394440e-03,
       -7.60530621e-04, -5.73858634e-04, -4.56143452e-04, -2.24340903e-03,
       -1.12518431e-03,  2.44126852e-04,  1.65269012e-03,  9.41634481e-04,
       -3.86359386e-03, -2.36559607e-03,  4.84643595e-04, -5.13243797e-05,
       -9.08409368e-04, -3.25027084e-03, -2.15616540e-03, -1.80353671e-03,
        1.20712193e-03,  9.44236641e-04,  1.81946229e-03, -2.40029478e-03,
        2.58515354e-04, -2.41504776e-05, -1.56854187e-03, -4.23695461e-03,
       -3.66646664e-03,  

In [36]:
p = pv.Plotter()
p.add_points(np.hstack([pts, z[:, np.newaxis]]))
p.add_mesh(surfaces[0].vtk)
p.show()

Widget(value="<iframe src='http://localhost:50953/index.html?ui=P_0x1d486e5b710_7&reconnect=auto' style='width…